# Visualization with ArcGIS smart mapping

## Smart mapping of line features

In [1]:
from arcgis.gis import *
gis = GIS()

In [2]:
map1 = gis.map('USA', 3)
map1

MapView(layout=Layout(height='400px', width='100%'), zoom=3.0)

In [3]:
search_result = gis.content.search('title:USA Freeway System AND owner:esri',
                                   item_type = 'Feature Layer')
search_result

[<Item title:"USA Freeway System" type:Feature Layer Collection owner:esri>]

In [4]:
freeway_item = search_result[0]
map1.add_layer(freeway_item)

### Display the fields

In [5]:
freeway_item.layers

[<FeatureLayer url:"https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Freeway_System/FeatureServer/1">,
 <FeatureLayer url:"https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Freeway_System/FeatureServer/2">]

In [6]:
# Use the first layer available on the service.
freeway_feature_layer = freeway_item.layers[0]

# Print the layer's url and field names.
print(freeway_feature_layer.url)
for field in freeway_feature_layer.properties['fields']:
    print(field['name'] + " \t " + field['type'])

https://services.arcgis.com/P3ePLMYs2RVChkJx/arcgis/rest/services/USA_Freeway_System/FeatureServer/1
OBJECTID 	 esriFieldTypeOID
ROUTE_NUM 	 esriFieldTypeString
CLASS 	 esriFieldTypeString
NUMBER 	 esriFieldTypeString
SUFFIX 	 esriFieldTypeString
DIST_MILES 	 esriFieldTypeDouble
DIST_KM 	 esriFieldTypeDouble


In [7]:
map1.add_layer(freeway_feature_layer, {"renderer": "ClassedSizeRenderer", "field_name": "DIST_MILES"})

In [8]:
# Change map's extent to San Diego, CA
map1.center = [32.715, -117.1]

In [9]:
map1.zoom = 10

# Visualizing area features

## Use definition queries to limit data during rendering

In [10]:
map2 = gis.map('California', 6)
map2

MapView(layout=Layout(height='400px', width='100%'), zoom=6.0)

In [11]:
map3 = gis.map('Seattle, WA', 6)
map3

MapView(layout=Layout(height='400px', width='100%'), zoom=6.0)

In [12]:
map3.add_layer({"type": "FeatureLayer",
                "url": "//sampleserver6.arcgisonline.com/arcgis/rest/services/Census/MapServer/2",
                "definition_expression": "STATE_NAME='Washington'",
                "renderer": "ClassedColorRenderer",
                "field_name": "POP2007",
                "opacity": 0.7
               })

### Visualizing location data

In [13]:
map4 = gis.map('Los Angeles', 8)
map4

MapView(layout=Layout(height='400px', width='100%'), zoom=8.0)

In [14]:
map4.add_layer({"type": "FeatureLayer",
                "url": "http://services1.arcgis.com/hLJbHVT9ZrDIzK0I/arcgis/rest/services/EQMagGt4/FeatureServer/0",
                "renderer": "HeatmapRenderer",
                "opacity": 0.75
               })

### Property Updating

In [15]:
from arcgis.gis import GIS
gis = GIS()
agol_gis = GIS()

def print_map_info(map):
    print("Mode =\t\t{}".format(map.mode))
    if map.mode == "2D":
        print("Zoom =\t\t{}\n".format(map.zoom) + \
             "Rotation =\t{}".format(map.rotation))
    elif map.mode == "3D":
        print("Zoom =\t\t{}\n".format(map.zoom) + \
             "Tilt =\t\t{}\n".format(map.tilt) + \
             "Heading = \t{}".format(map.heading))
    else:
        raise Exception("Not supported argument.")

usa_map = gis.map("USA")
usa_map.mode = "3D"
usa_map

MapView(layout=Layout(height='400px', width='100%'), mode='3D')

In [16]:
print_map_info(usa_map)

Mode =		3D
Zoom =		-1.0
Tilt =		0.0
Heading = 	0.0


### Callbacks

In [17]:
from arcgis.gis import GIS
import arcgis.geocoding as geocoding

callback_map = gis.map('San Diego convention center, San Diego, CA', 16)

def find_addr(callback_map, g):
    try:
        callback_map.draw(g)
        geocoded = geocoding.reverse_geocode(g)
        print(geocoded['address']['Match_addr'])
    except:
        print("Couldn't match address. Try another place...")

callback_map.on_click(find_addr)
callback_map

MapView(layout=Layout(height='400px', width='100%'), zoom=16.0)

Petco Park


### 3D Feature Layers

Let's create a new map of Zion National Park in Utah.
3D mode will show the elevation of the many canyons, rides, and elevation changes.
It will even cast shadows.

In [18]:
from arcgis.gis import GIS
zion_map = gis.map("Angel's Landing UT", 15)
zion_map.mode = "3D"
zion_map.tilt = 40
zion_map.heading = 180

zion_map

MapView(heading=180.0, layout=Layout(height='400px', width='100%'), mode='3D', tilt=40.0, zoom=15.0)

In [19]:
trails_layer = agol_gis.content.get('dd0889d7ccd340dd876dac12184e99f9').layers[0]
zion_map.add_layer(trails_layer)

### Autocasting

In [20]:
symbol = {"type": "line-3d", # JS type of new LineSymbol3D()
         "symbolLayers": [{
             "type": "path", # JS type of new PathSymbol3DLayer()
             "size": 20, # 20 meters in diameter
             "material": {
                 "color": "#0083ff" # The hex color mode
             }
         }]
}

zion_map.add_layer(trails_layer,
                   {"renderer": "autocast", # Tell Python to autocast JS types
                   "type": "simple", # JS type of SimpleRenderer()
                    "symbol": symbol # The symbol we previously defined
                   }
)

### Taking advantage of Autocasting

In [21]:
from arcgis.features import FeatureLayer
anon_gis = GIS()

hurricane_layer = FeatureLayer(
    url="https://sampleserver6.arcgisonline.com/arcgis/rest/services/Hurricanes/MapServer/0"
)
hurricane_map = anon_gis.map('Atlantic Ocean', 3)
hurricane_map.add_layer(hurricane_layer)
hurricane_map

MapView(layout=Layout(height='400px', width='100%'), zoom=3.0)